In [ ]:
%matplotlib inline
import sys
from os.path import dirname, abspath

# Paths for this analysis

In [ ]:
model_path = abspath("../trained_models/5b/a1ad79e6060e26746af38c/1.pkl")
training_data = abspath("../data/training_data_lower_atmos.nc")
subset_path = 'subset.nc'
scm_output = 'scm.nc'
sam_nn_2d = 'test/OUT_2D/CASE__1.2Dbin_1.nc'
sam_nn_3d = 'test/OUT_3D/*.nc'

In [ ]:
%cd {dirname(model_path)}

# Analysis

Save a subset of the data for running the column model:

In [ ]:
xr.open_dataset(training_data).isel(x=slice(0,128), y=slice(32,33), time=slice(0,200))\
.to_netcdf(subset_path)

In [ ]:
!rm -f {scm_output}
%run -m uwnet.columns {model_path} {subset_path} {scm_output}

In [ ]:
scm_output_ds = xr.open_dataset(scm_output)

## Single column model output

Humidity

In [ ]:
scm_output_ds.QT[0,:,0,:].plot.contourf(y='z', vmax=25)

In [ ]:
scm_output_ds.FQTNN[0,:,0,:].plot.contourf(y='z')

In [ ]:
time =scm_output_ds.SLI.time 
Q1 = (scm_output_ds.SLIOBS.shift(time=-1)-scm_output_ds.SLIOBS)/(time.shift(time=-1)-time)/86400 - scm_output_ds.FSLI 

In [ ]:
Q1[0,:,0,:].plot(vmax=.001, vmin=-.001, cmap='RdBu_r')

In [ ]:
scm_output_ds.FSLINN[0,:,0,:].plot(vmax=.001, vmin=-.001, cmap='RdBu_r')

The predicted heating is drastically wrong. This of course doesn't account for the constraint enforcing layer

In [ ]:
scm_output_ds.FSLINN[0,:,0,:].plot.contourf(y='z')

The inferred heating seems all wrong.

In [ ]:
scm_output_ds.SLI[:,:,0,0].plot(y='z')

In [ ]:
scm_output_ds.SLIOBS[:,:,0,0].plot(y='z')

# Run with SAM

In [ ]:
!rm -rf test && {sys.executable} -m src.criticism.run_sam_ic_nn {model_path} 2> err > out

In [ ]:
!tail err

In [ ]:
ds_2d = xr.open_mfdataset(sam_nn_2d)
ds_3d = xr.open_mfdataset(sam_nn_3d)

Here is a zonal slice of the neural networks predicted source of humidity

In [ ]:
ds_3d.FQTNN[-1,:,32,:].plot()

I suspect the time stepper is mostly ignoring the source of humidity about a few thousand feet, where the negative humidity applying constraint must be applied.

And of FSLNN

In [ ]:
ds_2d.W500.mean('x').plot()

In [ ]:
ds_3d.FQTNN[::4,15].plot(col='time', col_wrap=3)